In [212]:
# import usual things
import pandas as pd
import bqplot
import numpy as np
import traitlets
import ipywidgets
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline

In [213]:
buildings=pd.read_csv("building_inventory.csv",
                     na_values={
                         'Year Acquired': 0,
                         "Year Constructed": 0,
                         "Square Footage" : 0
                         
                     })

In [214]:
building_ht=pd.pivot_table(buildings,
                           index=['Congress Dist'],
                            values=['Square Footage'],
                            columns=['Agency Name'],
                            aggfunc=np.sum)
                        

In [215]:
building_ht_log=np.log10(building_ht)

In [216]:
building_ht

Square Footage  \
Agency Name   Appellate Court / Fifth District   
Congress Dist                                    
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
5                                          NaN   
6                                          NaN   
7                                          NaN   
8                                          NaN   
9                                          NaN   
10                                         NaN   
11                                         NaN   
12                                     15124.0   
13                                         NaN   
14                                         NaN   
15                                         NaN   
16                                         NaN   
17                                         NaN   
18                                         NaN   

                                                 \
Agency Name   Appellate Court / Fourth District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                           NaN   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                      16400.0   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                 \
Agency Name   Appellate Court / Second District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                       43330.0   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                          NaN   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                                         \
Agency Name   Appellate Court / Third District Chicago State University   
Congress Dist                                                             
0                                          NaN                      NaN   
1                                          NaN                1219492.0   
2                                          NaN                      NaN   
3                                          NaN                      NaN   
4                                          NaN                      NaN   
5                                          NaN                      NaN   
6         

In [217]:
building_ht.iat[0,6]

231278.0

In [218]:
agency_list=building_ht_log.columns.levels[1].to_list()

In [231]:
i,j=10,17
mySelectedLabel = ipywidgets.Label()
def on_selected(change):
        if len(change['owner'].selected) == 1:
        #print(change['owner'].selected[0])
            
            i, j = change['owner'].selected[0]
            
           
            mySelectedLabel.value = 'agency is' + agency_list[i]+', district is '+str(j)+'value is '+str(building_ht.iat[i,j])
            
            agency=agency_list[i]
            congress=j
            mask1=buildings['Agency Name']==agency
            mask2=buildings['Congress Dist']==congress
            mask=mask1&mask2
            target=buildings[mask]
            dic=defaultdict(int)
            for index, row in target.iterrows():
                dic[row["Year Acquired"]]+=row["Square Footage"]
        
            a=[]
            for d in dic:
                if d>=1300 and d<=2030:
                    a.append([d,dic[d]])
            a.sort()
            x=[]
            y=[]
            for a1,a2,in a:
                x.append(a1)
                y.append(a2)
            lines.x=x
            lines.y=y



In [232]:
agency_list

['Appellate Court / Fifth District',
 'Appellate Court / Fourth District',
 'Appellate Court / Second District',
 'Appellate Court / Third District',
 'Chicago State University',
 'Department of Agriculture',
 'Department of Central Management Services',
 'Department of Corrections',
 'Department of Human Services',
 'Department of Juvenile Justice',
 'Department of Military Affairs',
 'Department of Natural Resources',
 'Department of Public Health',
 'Department of Revenue',
 'Department of State Police',
 'Department of Transportation',
 "Department of Veterans' Affairs",
 'Eastern Illinois University',
 "Governor's Office",
 'Governors State University',
 'Historic Preservation Agency',
 'IL State Board of Education',
 'Illinois Board of Higher Education',
 'Illinois Community College Board',
 'Illinois Courts',
 'Illinois Emergency Management Agency',
 'Illinois Medical District Commission',
 'Illinois State University',
 'Northeastern Illinois University',
 'Northern Illinois Uni

In [233]:

agency=agency_list[i]
congress=j
mask1=buildings['Agency Name']==agency
mask2=buildings['Congress Dist']==congress
mask=mask1&mask2
target=buildings[mask]
dic=defaultdict(int)
for index, row in target.iterrows():
    dic[row["Year Acquired"]]+=row["Square Footage"]
        
a=[]
for d in dic:
    if d>=1300 and d<=2030:
        a.append([d,dic[d]])
a.sort()
x=[]
y=[]
for a1,a2,in a:
    x.append(a1)
    y.append(a2)
        
x_sc = bqplot.LinearScale()
y_sc = bqplot.LogScale()

lines = bqplot.Lines(x=x, y=y, scales = {'x':x_sc, 'y':y_sc})
ax_x = bqplot.Axis(scale=x_sc, label='X Value')
ax_y = bqplot.Axis(scale=y_sc, label='Y Value', orientation='vertical')
fig_l = bqplot.Figure(marks=[lines], axes=[ax_x, ax_y])


In [234]:
col_sc = bqplot.ColorScale(scheme="Purples") 
# this is because the "bins" are just bins -- their order is NOT numerically important
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# 3. Axis -- for colors, the axis is a colorbar!
ax_col = bqplot.ColorAxis(scale = col_sc, orientation='vertical', side='right') 
ax_x = bqplot.Axis(scale = x_sc) # same x/y ax we had before
ax_y = bqplot.Axis(scale = y_sc, orientation='vertical')

# 4. Mark -- heatmap
heat_map = bqplot.GridHeatMap(color =building_ht_log, scales = {'color':col_sc, 'row':y_sc, 'column':x_sc},
                             interactions={'click':'select'},
                             anchor_style={'fill':'blue'},
                             selected_style={'opacity':1.0},
                             unselected_style={'opacity':0.8})
heat_map.observe(on_selected, 'selected')
fig = bqplot.Figure(marks = [heat_map], axes=[ax_col, ax_x, ax_y],size=(15,15))

In [235]:
figures = ipywidgets.VBox([mySelectedLabel, fig])
myDashboard = ipywidgets.HBox([figures, fig_l])

In [236]:
myDashboard

I used a hashmap to calculate the total footage for a selected area. I also use pivot_table function to transform the original 
data into an table
Also, I transfrom the year into log scale

I set the NaN to zero at first.
I used purple color because I like purple.


